# Osyczka and Kundu function multi-objective optimization (min).
---
Description:

- Optimization (min)
- Multi-objective (2)
- Constraints (6)
---

Minimize the equations given by:

\begin{cases}
      f_{1}\left(\mathbf{x}\right) = -(25(x_1 - 2)^2 + (x_2 - 2)^2 + (x_3 - 1)^2 + (x_4 - 4)^2 + (x_5 - 1)^2), \\
      f_{2}\left(\mathbf{x}\right) = x_1^2 + x_2^2 + x_3^2 + x_4^2 + x_5^2 + x_6^2, \\
\end{cases}

subject to:

\begin{cases}
      C_{1}\left(\mathbf{x}\right) = (x_1 + x_2 − 2) \geq 0, \\
      C_{2}\left(\mathbf{x}\right) = (6 - x_1 − x_2) \geq 0, \\
      C_{3}\left(\mathbf{x}\right) = (2 - x_2 + x_1) \geq 0, \\
      C_{4}\left(\mathbf{x}\right) = (2 − x_1 + 3x_2) \geq 0, \\
      C_{5}\left(\mathbf{x}\right) = (4 − (x_3 − 3)2 − x_4) \geq 0, \\
      C_{6}\left(\mathbf{x}\right) = ((x_5 − 3)^2 + x_6−4) \geq 0, \\
\end{cases}

where:

\begin{cases}
      0\le x_1, x_2, x_6 \le 10, \\
      1\le x_3, x_5 \le 5, \\
      0\le x_4 \le 6. \\
\end{cases}

The Pareto-optimal region is a concatenation of five regions. Every region lies on some of the constraints. However, for the entire Pareto-optimal region, $\hat{x}_4=\hat{x}_6=0$.

### First we import python libraries and set up the directory of our code.

In [3]:
import os, sys
import numpy as np
from math import fsum

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [5]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.island_model_ga import IslandModelGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.random_mutator import RandomMutator

# Import Migration Operator(s).
from pygenalgo.operators.migration.clockwise_migration import ClockwiseMigration

### Define the multi-objective function, which plays also the role of the 'fitness' function.

In addition, we define a 'rand_fx', for each variable, which takes the role of the 'random()' method
of the Genes, using the required range.

In [7]:
# Multi-objective function.
def fun_OSY(individual: Chromosome, f_min: bool = True):
    
    # Extract the gene values.
    x = [gene.value for gene in individual.genome]
        
    # Compute each objective function.
    f1 = -(25.0*(x[0] - 2.0)**2 +
                (x[1] - 2.0)**2 +
                (x[2] - 1.0)**2 +
                (x[3] - 4.0)**2 +
                (x[4] - 1.0)**2)
    
    f2 = fsum([i**2 for i in x])
    
    # Compute the constraints.
    C1 = min(0.0, (x[0] + x[1] - 2.0))**2
    C2 = min(0.0, (6.0 - x[0] - x[1]))**2
    C3 = min(0.0, (2.0 + x[0] - x[1]))**2
    C4 = min(0.0, (2.0 - x[0] + 3.0*x[1]))**2
    C5 = min(0.0, (4.0 - (x[2] - 3.0)**2 - x[3]))**2
    C6 = min(0.0, ((x[4] - 3.0)**2 + x[5] - 4.0))**2
    
    # Compute the final value.
    f_val = fsum([0.35*f1, 0.65*f2, C1, C2, C3, C4, C5, C6])
    
    # Return the negative (to account for minimization).
    return -f_val if f_min else f_val
# _end_def_

# Random functions (one for each variable):
rand_fx1 = lambda: np.random.uniform(0.0, 10.0)
rand_fx2 = lambda: np.random.uniform(0.0, 10.0)
rand_fx3 = lambda: np.random.uniform(1.0,  5.0)
rand_fx4 = lambda: np.random.uniform(0.0,  6.0)
rand_fx5 = lambda: np.random.uniform(1.0,  5.0)
rand_fx6 = lambda: np.random.uniform(0.0, 10.0)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each gene has different random() function, because they are valid in different ranges according to the 
problem definition.

In [9]:
# Define the number of chromosomes.
N = 400

# Initial population.
population = [Chromosome([Gene(np.random.uniform(0.0, 10.0), rand_fx1),
                          Gene(np.random.uniform(0.0, 10.0), rand_fx2),
                          Gene(np.random.uniform(1.0,  5.0), rand_fx3),
                          Gene(np.random.uniform(0.0,  6.0), rand_fx4),
                          Gene(np.random.uniform(1.0,  5.0), rand_fx5),
                          Gene(np.random.uniform(0.0, 10.0), rand_fx6)], np.nan, True)
              for _ in range(N)]

# Create the IslandModelGA object that will carry on the optimization.
test_GA = IslandModelGA(initial_pop=population,
                        fit_func=fun_OSY, num_islands=4,
                        select_op=LinearRankSelector(),
                        mutate_op=RandomMutator(0.10),
                        crossx_op=UniformCrossover(0.8),
                        migrate_op=ClockwiseMigration())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [11]:
test_GA(epochs=1000, elitism=True, f_tol=1.0e-8, allow_migration=True, verbose=True)


Current period 1 / 10:

Best Fitness in island 0 is:= 455.27007.
Best Fitness in island 1 is:= 449.50585.
Best Fitness in island 2 is:= 454.22688.
Best Fitness in island 3 is:= 453.81357.

Current period 2 / 10:

Best Fitness in island 0 is:= 455.38295.
Best Fitness in island 1 is:= 455.41824.
Best Fitness in island 2 is:= 454.44032.
Best Fitness in island 3 is:= 454.44055.

Current period 3 / 10:

Best Fitness in island 0 is:= 455.40365.
Best Fitness in island 1 is:= 455.80687.
Best Fitness in island 2 is:= 455.49225.
Best Fitness in island 3 is:= 456.56091.

Current period 4 / 10:

Best Fitness in island 0 is:= 456.71694.
Best Fitness in island 1 is:= 455.80687.
Best Fitness in island 2 is:= 455.81403.
Best Fitness in island 3 is:= 456.71535.

Current period 5 / 10:

Best Fitness in island 0 is:= 456.72918.
Best Fitness in island 1 is:= 456.76264.
Best Fitness in island 2 is:= 455.82630.
Best Fitness in island 3 is:= 456.76606.

Current period 6 / 10:

Best Fitness in island 0 is:= 

In [12]:
x = [gene.value for gene in test_GA.best_chromosome().genome]

f1 = -(25.0*(x[0] - 2.0)**2 +
            (x[1] - 2.0)**2 +
            (x[2] - 1.0)**2 +
            (x[3] - 4.0)**2 +
            (x[4] - 1.0)**2)

f2 = fsum([i**2 for i in x])

for i, xi in enumerate(x, start=1):
    print(f"x{i}={xi:.5f}")

print(" ")
print(f"f1(x) = {f1:.5f}")
print(f"f2(x) = {f2:.5f}")

x1=9.99931
x2=1.87338
x3=1.00022
x4=0.00040
x5=1.00070
x6=0.00385
 
f1(x) = -1615.73693
f2(x) = 105.49761


### End of file